In [ ]:
import openai
import json
import re
import pandas as pd
import requests
import time
from google.colab import userdata

df = pd.read_csv('/content/all_data.csv')

df = df[
    (df['action_type'].isna() == False) &
    (df['self-typical-confusion'].isna() == False) &
    (df['self-typical-interactions'].isna() == False) &
    (df['self-typical-confusion'] > 2) &
    (df['self-typical-interactions'] > 2)  &
    (df['state'].isna() == False)
].reset_index(drop=True)

columns = [
    'question',
    'qid',
    'scenario',
    'ground_truth',
    'student_incorrect_solution',
    'student_profile',
    'conversation',
    'state',
    'action_text',
    'action_type',
    'done',
    'self-typical-confusion',
    'self-typical-interactions'
]

new_df = df[columns]

state_df = new_df.copy()
state_df['full_state'] = None

for i in range(0, len(new_df)):
  if 'START' in str(new_df.iloc[i]['state']):
    state_df.at[i, 'full_state'] = ['', 'START']
    continue

  prev_action_text = new_df.iloc[i-1]['action_text']
  curr_state = new_df.iloc[i]['state']

  state_df.at[i, 'full_state'] = [prev_action_text, curr_state]

state_df

state_df['conversation']
curr_pos = 1
if len(state_df) > 0:
  state_df.loc[0, 'convo_turn'] = curr_pos

for i in range(1, len(state_df)):
    # check if a new conversation starts (0 follows a 1)
    if state_df.loc[i-1, 'done'] == 1 and state_df.loc[i, 'done'] == 0:
        # Reset position counter for new conversation
        curr_pos = 1
    else:
        curr_pos += 1

    # assign the current position within conversation
    state_df.loc[i, 'convo_turn'] = curr_pos * 1

state_df['convo_turn'] = state_df['convo_turn'].astype(int)
state_df['old_state'] = state_df['state']
state_df['state'] = state_df['full_state']
state_df['next_action'] = state_df['action_text']

state_df
columns = [
    'question',
    'qid',
    'scenario',
    'ground_truth',
    'state',
    'next_action',
    'convo_turn',
    'action_type',
    'done'
    ]

state_df = state_df[columns]

In [ ]:
state_df

In [ ]:
import json
import re
import time
import os
import requests
import pandas as pd
from pathlib import Path


def extract_json(text):
    try:
        data = json.loads(text)
        if _validate_schema(data):
            return data
    except json.JSONDecodeError:
        pass

    json_pattern = r'\s*(\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\})\s*'
    for potential_json in reversed(re.findall(json_pattern, text, re.DOTALL)):
        try:
            data = json.loads(potential_json)
            if _validate_schema(data):
                return data
        except json.JSONDecodeError:
            continue

    return None


def _validate_schema(data):
    if not isinstance(data, dict) or {'state', 'action'} - data.keys():
        return False

    state, action = data['state'], data['action']

    # validate state fields
    for k in ('listen_to_feedback', 'problem_progress', 'correct_solution'):
        if not isinstance(state.get(k), int):
            return False

    if (state['listen_to_feedback'] not in (0, 1) or
            not 0 <= state['problem_progress'] <= 100 or
            state['correct_solution'] not in (0, 1)):
        return False

    # validate action fields
    if (not isinstance(action.get('next_action_hint_strength'), int) or
            not 0 <= action['next_action_hint_strength'] <= 4):
        return False

    # validate category/subcategory combinations
    cat = str(action.get('next_action_category', '')).lower().strip()
    sub = str(action.get('next_action_subcategory', '')).lower().strip()

    allowed = {
        'focus': {'seek next step', 'confirm calculation',
                 're-direct to sub-problem', 'highlight missing info'},
        'probing': {'ask for explanation', 'seek self-correction',
                   'hypothetical variation', 'check understanding/concept',
                   'encourage comparison'},
        'telling': {'partial reveal (strategy)', 'full reveal (answer)',
                   'corrective explanation'},
        'generic': {'greeting/farewell', 'acknowledgment/praise',
                   'summarize progress', 'general inquiry/filler'},
    }

    return cat in allowed and sub in allowed[cat]


def process_row(idx, row, state_df, checkpoint, api_key):
    question = row.get('question', '')
    correct_solution = row.get('ground_truth', '')
    student_profile = row.get('student_profile', '')
    current_state = row.get('state', '')
    action_text = row.get('action_text', '')
    action_type = str(row.get('action_type', '')).lower()
    is_start = isinstance(current_state, str) and 'START' in current_state

    prompt = build_prompt(question, correct_solution, student_profile,
                         current_state, action_text, action_type, is_start)

    payload = {
        'model': 'gpt-4.1-mini-2025-04-14',
        'messages': [{'role': 'user', 'content': prompt}],
        'max_tokens': 400
    }
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    retry_request(idx, payload, headers, state_df, is_start, checkpoint)

def process_batch(todo_df, state_df, checkpoint, api_key, delay=3):
    total_remaining = len(todo_df)
    print(f"{total_remaining} rows need processing")

    for counter, (idx, row) in enumerate(todo_df.iterrows(), 1):
        if idx not in state_df.index:
            continue

        if counter > 1:
            time.sleep(delay)

        print(f"[{counter}/{total_remaining}] Row {idx}: sending request")
        process_row(idx, row, state_df, checkpoint, api_key)

def build_prompt(question, correct_solution, student_profile, current_state,
                action_text, action_type, is_start):
    return f"""
You are analyzing math tutoring sessions to extract features for an AI tutor.

CONTEXT:
- Question: {question}
- Correct solution: {correct_solution}
- Student profile: {student_profile}
- Current conversation (teacher feedback, student response): {current_state}
- Next action text: {action_text}
- Next action type: {action_type}
- Is start of conversation: {is_start}

TASK:
Extract the following features from this tutoring interaction:

1. LISTEN TO FEEDBACK:
   - 1 if the student has incorporated the teacher's feedback in the current conversation state
   - 0 if not

2. PROBLEM PROGRESS - Numerical estimate (0-100) of how close, given the student's current conversation state, the student is to the correct solution. For example:
  - 0: No progress/completely wrong approach/conversation just starting
  - 25: Beginning to understand but major errors remain
  - 50: Halfway to solution with some key insights
  - 75: Minor errors but mostly correct approach
  - 100: Complete and correct solution

  IMPORTANT: Carefully analyze the current conversation state to determine progress.
  At the start of a conversation with no student responses yet, progress should be 0.
  (You may use any value between 0-100, not just the examples above)

3. CORRECT SOLUTION:
   - 0 if the student's final answer is incorrect
   - 1 if correct

4. NEXT ACTION HINT STRENGTH (0-4):
   How strong is the teacher's hint for the next action? (0=very minimal hint, 4=essentially giving away the entire solution)

5. NEXT ACTION SUBTYPE:
    Choose exactly one of the following, depending on the next action type:

    If action type is 'focus':
    - 'Seek Next Step'
    - 'Confirm Calculation'
    - 'Re-direct to Sub-Problem'
    - 'Highlight Missing Info'

    If action type is 'probing':
    - 'Ask for Explanation'
    - 'Seek Self-Correction'
    - 'Hypothetical Variation'
    - 'Check Understanding/Concept'
    - 'Encourage Comparison'

    If action type is 'telling':
    - 'Partial Reveal (Strategy)'
    - 'Full Reveal (Answer)'
    - 'Corrective Explanation'

    If action type is 'generic':
    - 'Greeting/Farewell'
    - 'Acknowledgment/Praise'
    - 'Summarize Progress'
    - 'General Inquiry/Filler'

FORMAT YOUR RESPONSE AS JSON:
{{
  'state': {{
    'listen_to_feedback': 0 or 1,
    'problem_progress': integer_between_0_and_100,
    'correct_solution': 0 or 1,
    'reasoning': briefly_explain_why_student_is_or_isnt_correct
  }},
  'action': {{
    'next_action_hint_strength': integer_between_0_and_4,
    'next_action_category': '{action_type}',
    'next_action_subcategory': 'pick_the_exact_one_subcategory_name'
  }}
}}

IMPORTANT:
- Return ONLY the JSON object with no additional explanation
- All numeric fields must be valid integers
""".strip()


def retry_request(idx, payload, headers, state_df, is_start, checkpoint):
    retry, max_retries = 0, 3

    while retry < max_retries:
        try:
            resp = requests.post(
                'https://api.openai.com/v1/chat/completions',
                headers=headers,
                json=payload
            )

            if resp.status_code == 429:
                print('Rate‑limit hit, sleeping 60 seconds')
                time.sleep(60)
                continue

            resp.raise_for_status()
            out = resp.json()['choices'][0]['message']['content']
            state_df.at[idx, 'raw_model_response'] = out

            data = extract_json(out)
            if not data:
                raise ValueError('schema fail')

            update_dataframe(idx, data, state_df, is_start)
            print(f"Row {idx} processed successfully")
            break

        except Exception as e:
            retry += 1
            print(f"Row {idx} error ({retry}/{max_retries}): {e}")

            if retry == max_retries:
                state_df.at[idx, 'extraction_failed'] = True
                state_df.at[idx, 'error_message'] = str(e)
            else:
                time.sleep(2)

    # save checkpoint after each row
    tmp = checkpoint + '.tmp'
    state_df.to_csv(tmp, index=False)
    os.replace(tmp, checkpoint)
    print(f"Checkpoint saved after row {idx}")


def update_dataframe(idx, data, state_df, is_start):
    s, a = data['state'], data['action']

    state_df.at[idx, 'listen_to_feedback'] = s['listen_to_feedback']

    progress = 0 if is_start and s['problem_progress'] > 25 else s['problem_progress']
    state_df.at[idx, 'problem_progress'] = progress

    state_df.at[idx, 'correct_solution'] = s['correct_solution']
    state_df.at[idx, 'next_action_hint_strength'] = a['next_action_hint_strength']
    state_df.at[idx, 'next_action_category'] = a['next_action_category']
    state_df.at[idx, 'next_action_subcategory'] = a['next_action_subcategory']


def main():
    # Configuration
    checkpoint = 'part1_in_progress.csv'
    output_file = 'part1_mathdial_complete.csv'
    requests_per_minute = 20
    delay_between_requests = 60 / requests_per_minute

    api_key = os.environ.get('OPENAI_API_KEY')
    if not api_key:
        raise ValueError('OPENAI_API_KEY missing from environment variables')

    if Path(checkpoint).exists():
        state_df = pd.read_csv(checkpoint)
        print(f"Loaded checkpoint with {len(state_df)} rows")
    else:
        raise FileNotFoundError(f'Checkpoint file {checkpoint} not found')

    required_columns = [
        'raw_model_response',
        'listen_to_feedback',
        'problem_progress',
        'correct_solution',
        'next_action_hint_strength',
        'next_action_category',
        'next_action_subcategory',
        'extraction_failed',
        'retry_count',
        'error_message'
    ]

    for col in required_columns:
        if col not in state_df.columns:
            state_df[col] = pd.NA

    todo_df = state_df[state_df['raw_model_response'].isna()].copy()
    process_batch(todo_df, state_df, checkpoint, api_key, delay_between_requests)

    state_df.to_csv(output_file, index=False)

if __name__ == '__main__':
    main()

Below: Takes the tutoring actions returned by GPT-4.1 and maps it to 80 unique action IDs.

In [ ]:
# action apping dictionary (1-based hint strengths).
ACTION_MAP = {
  'focus': {
    'Seek Next Step': {
      '1': 0,
      '2': 1,
      '3': 2,
      '4': 3,
      '5': 4
    },
    'Confirm Calculation': {
      '1': 5,
      '2': 6,
      '3': 7,
      '4': 8,
      '5': 9
    },
    'Re-direct to Sub-Problem': {
      '1': 10,
      '2': 11,
      '3': 12,
      '4': 13,
      '5': 14
    },
    'Highlight Missing Info': {
      '1': 15,
      '2': 16,
      '3': 17,
      '4': 18,
      '5': 19
    }
  },
  'probing': {
    'Ask for Explanation': {
      '1': 20,
      '2': 21,
      '3': 22,
      '4': 23,
      '5': 24
    },
    'Seek Self-Correction': {
      '1': 25,
      '2': 26,
      '3': 27,
      '4': 28,
      '5': 29
    },
    'Hypothetical Variation': {
      '1': 30,
      '2': 31,
      '3': 32,
      '4': 33,
      '5': 34
    },
    'Check Understanding/Concept': {
      '1': 35,
      '2': 36,
      '3': 37,
      '4': 38,
      '5': 39
    },
    'Encourage Comparison': {
      '1': 40,
      '2': 41,
      '3': 42,
      '4': 43,
      '5': 44
    }
  },
  'telling': {
    'Partial Reveal (Strategy)': {
      '1': 45,
      '2': 46,
      '3': 47,
      '4': 48,
      '5': 49
    },
    'Full Reveal (Answer)': {
      '1': 50,
      '2': 51,
      '3': 52,
      '4': 53,
      '5': 54
    },
    'Corrective Explanation': {
      '1': 55,
      '2': 56,
      '3': 57,
      '4': 58,
      '5': 59
    }
  },
  'generic': {
    'Greeting/Farewell': {
      '1': 60,
      '2': 61,
      '3': 62,
      '4': 63,
      '5': 64
    },
    'Acknowledgment/Praise': {
      '1': 65,
      '2': 66,
      '3': 67,
      '4': 68,
      '5': 69
    },
    'Summarize Progress': {
      '1': 70,
      '2': 71,
      '3': 72,
      '4': 73,
      '5': 74
    },
    'General Inquiry/Filler': {
      '1': 75,
      '2': 76,
      '3': 77,
      '4': 78,
      '5': 79
    }
  }
}

def get_next_action_id(row):
    cat = row['next_action_category']
    sub = row['next_action_subcategory']
    hint_str = str(row['next_action_hint_strength'] + 1)
    return ACTION_MAP[cat][sub][hint_str]

df = pd.read_csv('/content/part1_mathdial_complete.csv')
df